# Introduction to SQL for Excel Users – Part 9: Basic Windows

[Original post](https://www.daveondata.com/blog/introduction-to-sql-for-excel-users-part-9-basic-windows/)

## Windows of Data in Excel

A window of data is a logical concept that we use all the time – even if we don’t know the formal name for it.

A window of data is a logical subset of the data in a table.

One way to think of this idea is in the context of groups.

In the last post I covered “hard groups” in both Excel and SQL.

When using “hard groups” in both technologies, the data is transformed as illustrated in the following Excel pivot table:

![pivot table 1](09\pivottable1.png)

In the pivot table ☝ the “hard groups” are AM, midnight, PM1, and PM2.

Conversely, one can think of windows as “soft groups” within the data.

Windows do not produce a transformation of the table.

In the following screenshot, I’ve illustrated some windows in the CallCenter data by color-coding the logical “soft groups”:

![excel windows 1](09\excelwindows1.png)

For each DateKey in the CallCenter data, there exists multiple rows of data.

That is, within the CallCenter data there exists logical sub-groupings of the data.

Windows of data allow you to work with these sub-groupings of data.

I will illustrate this concept by using Excel’s multi-level sort functionality as follows:

![call center sorting](09\callcentersorting.png)

Notice how the structure of the table does not change, but the order of the data has changed.

Conceptually, the first level of sorting defined a window in the data (i.e., a logical subset) and the second level of sorting then operated on the data within the window.

Here’s the logical process for working with windows of data using Excel:

1. Create windows of data (e.g., using multi-level sorting in Excel)
1. Define how the windows will be created (e.g., picking the first-level sorting in Excel)
1. Define some operation on the data in the windows (e.g., picking the second-level sorting in Excel)

Here’s a shocker – the above logical process is the same in SQL! 😉

## Windows of Data in SQL

NOTE – What you will see in this post is supported by major DBs like Oracle and PostgreSQL.

T-SQL supports windows of data via the OVER keyword and the definition of the windows using PARTITION BY.

T-SQL supports a number of window functions, but for this post we’re going to look at my favorite, ROW_NUMBER.

It’s best to see ROW_NUMBER in action to understand what it does – and why it is so awesome!

Let’s revisit our logical process using the above T-SQL keywords:

- Create windows of data using OVER
- Define how the windows will be created using PARTITION BY
- Define some operation on the data in the windows using ROW_NUMBER

OK, I’m going to put this all together for a first-pass SQL query:

```
-- this will produce an error
SELECT FCC.FactCallCenterID
      ,FCC.DateKey
      ,FCC.WageType
      ,FCC.Shift
      ,ROW_NUMBER() OVER (PARTITION BY FCC.DateKey) AS RowNum
FROM FactCallCenter FCC
```
Turns out we need to add one more thing to our logical process when using ROW_NUMBER:

- Create windows of data using OVER
- Define how the windows will be created using the PARTITION BY
- Define some operation on the data in the windows using ROW_NUMBER
- Define a sort order for the data in the windows using ORDER BY

OK, I’m going to update the query thusly:

In [ ]:
SELECT FCC.FactCallCenterID
      ,FCC.DateKey
      ,FCC.WageType
      ,FCC.Shift
      ,ROW_NUMBER() OVER (PARTITION BY FCC.DateKey ORDER BY FCC.Shift ASC) AS RowNum
FROM FactCallCenter FCC

NOTE – Always use an alias with window functions, otherwise the column will have no name.

Outstanding!

Notice what the mighty ROW_NUMBER does.

It allows you to define data windows, sort the data in the windows and then assign the row number based on all of this windowing goodness.

Using what I’ve covered in the series so far, I can ask some interesting questions of the data.

Lets say I’m the call center manager.

I might be curious to know for each day, which Shift was busiest in terms of the highest number of Calls.

The mighty ROW_NUMBER to the rescue!

Here’s the SQL:

In [ ]:
SELECT FCC.FactCallCenterID
      ,FCC.DateKey
      ,FCC.WageType
      ,FCC.Shift
      ,FCC.Calls
      ,ROW_NUMBER() OVER (PARTITION BY FCC.DateKey ORDER BY FCC.Calls DESC) AS RowNum
FROM FactCallCenter FCC
ORDER BY FCC.DateKey ASC

Here’s what the SQL is doing:

- It’s defining windows by DateKey
- Ordering the data in each window by Calls in descending order
- Applying ROW_NUMBER to the sorted data in each window

Since I’m sorting the windows by Calls in descending order, everywhere RowNum = 1 is the busiest shift in terms of Calls!

As awesome as all this is, it gets better.

Your windows can be composed of “soft groups” of “soft groups”…

## Fancy Data Windows in SQL

As with SQL’s GROUP BY, you can list multiple columns in PARTITION BY.

This allows you to create “soft groups” of “soft groups”.

Let’s say I’m that call center manager again and I am interested in seeing the busiest days by Calls, but windowed by WageType and Shift.

I can fire up SSMS and crank out the following SQL code to get me the data:

In [1]:
SELECT FCC.FactCallCenterID
      ,FCC.DateKey
      ,FCC.WageType
      ,FCC.Shift
      ,FCC.Calls
      ,ROW_NUMBER() OVER (PARTITION BY FCC.WageType, FCC.Shift ORDER BY FCC.Calls DESC) AS RowNum
FROM FactCallCenter FCC
ORDER BY FCC.WageType, FCC.Shift ASC

(120 rows affected)

Total execution time: 00:00:00.008

FactCallCenterID,DateKey,WageType,Shift,Calls,RowNum
69,20140518,holiday,AM,430,1
89,20140523,holiday,AM,429,2
9,20140503,holiday,AM,416,3
13,20140504,holiday,AM,376,4
101,20140526,holiday,AM,278,5
97,20140525,holiday,AM,269,6
73,20140519,holiday,AM,268,7
17,20140505,holiday,AM,245,8
41,20140511,holiday,AM,215,9
45,20140512,holiday,AM,209,10


Once again, everywhere RowNum = 1 is the data that I’m interested in.

Not surprisingly, SQL window functions are wildly useful for analytics pro – you use them all the time!

There will be more coverage of this awesomeness – stay tuned.

## The Learning Arc

In the next post I will branch off to briefly introduce common table expressions (CTEs).

I’ll cover CTEs as they are super useful when working with the mighty ROW_NUMBER.

The series is rapidly getting to the point where you can answer some really interesting question with your data.

Stay healthy and happy data sleuthing!